<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2, Notebook 1: Train Data Cleaning, EDA, & Model Prep

# Part 1 -- 

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns

import statsmodels.api as sm

from scipy import stats
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [77]:
# Reading in train data and printing example/head
#- had to move csv into main folder. was unable to read using file path for some reason
train_df = pd.read_csv ("train.csv")
train_df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [4]:
train_df.shape
#Checking the shape

(2051, 81)

In [5]:
# dtypes and nulls for each column.
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               2051 non-null   int64  
 1   PID              2051 non-null   int64  
 2   MS SubClass      2051 non-null   int64  
 3   MS Zoning        2051 non-null   object 
 4   Lot Frontage     1721 non-null   float64
 5   Lot Area         2051 non-null   int64  
 6   Street           2051 non-null   object 
 7   Alley            140 non-null    object 
 8   Lot Shape        2051 non-null   object 
 9   Land Contour     2051 non-null   object 
 10  Utilities        2051 non-null   object 
 11  Lot Config       2051 non-null   object 
 12  Land Slope       2051 non-null   object 
 13  Neighborhood     2051 non-null   object 
 14  Condition 1      2051 non-null   object 
 15  Condition 2      2051 non-null   object 
 16  Bldg Type        2051 non-null   object 
 17  House Style   

In [6]:
# renaming columns - replacing spaces with underscores
# and making lower case for easier access
train_df.columns = train_df.columns.str.lower().str.replace(' ', '_')

In [7]:
# Confirming column name changes.
train_df.columns[:5]

Index(['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage'], dtype='object')

In [8]:
# Referenced Pandas documentation on .any() to sort out only columns containing nulls.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.any.html
null_locations = train_df.columns[train_df.isnull().any()]
train_df[null_locations].isnull().sum()

lot_frontage       330
alley             1911
mas_vnr_type        22
mas_vnr_area        22
bsmt_qual           55
bsmt_cond           55
bsmt_exposure       58
bsmtfin_type_1      55
bsmtfin_sf_1         1
bsmtfin_type_2      56
bsmtfin_sf_2         1
bsmt_unf_sf          1
total_bsmt_sf        1
bsmt_full_bath       2
bsmt_half_bath       2
fireplace_qu      1000
garage_type        113
garage_yr_blt      114
garage_finish      114
garage_cars          1
garage_area          1
garage_qual        114
garage_cond        114
pool_qc           2042
fence             1651
misc_feature      1986
dtype: int64

In [9]:
# Correlation matrix for features vs. sale price.
train_df.corr()[['saleprice']].sort_values(by='saleprice', ascending=False)

,saleprice
saleprice,1.000000
overall_qual,0.800207
gr_liv_area,0.697038
garage_area,0.650270
garage_cars,0.648220
total_bsmt_sf,0.628925
1st_flr_sf,0.618486
year_built,0.571849
year_remod/add,0.550370
full_bath,0.537969


---
## Section 2

### Cleaning Training DataFrame


In [10]:
# I have dropped multiple columns that are primarily null values per the info above. 
# (over 1,000 nulls) 
train_df.drop(columns=['alley', 'pool_qc', 'fence', 'misc_feature'], inplace=True)

In [11]:
# Confirming change in shape / 4 less columns
train_df.shape

(2051, 77)

In [12]:
# function to fill null values in columns with the value/string 'None'.
def fill_nulls(features):
    # Iterating through the input list, 'features'.
    for feature in features:
        # Replacing null values with the text string 'None' for each feature in features, and saving to my dataframe.
        train_df[feature].replace(to_replace = np.nan, value = 'None', inplace = True)
    return train_df
# assistance within study group

In [13]:
# defining 'features'/columns and filling nulls wihtin them with above function
features = ['mas_vnr_type', 'bsmt_exposure', 'bsmt_qual', 'bsmt_cond', 'bsmtfin_type_1', 'bsmtfin_type_2',
            'garage_type', 'garage_finish', 'garage_qual', 'garage_cond', 'fireplace_qu']
fill_nulls(features)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,...,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,NaN,13517,Pave,IR1,Lvl,AllPub,...,44,0,0,0,0,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,IR1,Lvl,AllPub,...,74,0,0,0,0,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,Reg,Lvl,AllPub,...,52,0,0,0,0,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,Reg,Lvl,AllPub,...,0,0,0,0,0,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,IR1,Lvl,AllPub,...,59,0,0,0,0,0,3,2010,WD,138500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,1587,921126030,20,RL,79.0,11449,Pave,IR1,HLS,AllPub,...,276,0,0,0,0,0,1,2008,WD,298751
2047,785,905377130,30,RL,NaN,12342,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,0,3,2009,WD,82500
2048,916,909253010,50,RL,57.0,7558,Pave,Reg,Bnk,AllPub,...,0,0,0,0,0,0,3,2009,WD,177000
2049,639,535179160,20,RL,80.0,10400,Pave,Reg,Lvl,AllPub,...,189,140,0,0,0,0,11,2009,WD,144000


In [14]:
# Replacing nulls for masonry veneer area with a value of 0.
# As, according to data dictionary, null values for masonry venear type is 0 sq ft.
train_df['mas_vnr_area'].replace(to_replace = np.nan, value = 0, inplace = True)

In [15]:
# After referencing the data dictionary, I'm filling all remaining numeric nulls with the mean value of their column.
train_df.fillna(train_df.mean(), inplace = True)

In [16]:
# Defining a function, 'float_to_int' to accept one argument/a list of features.
def float_to_int(features2):
    # Iterating through the list of features.
    for feature in features2:
        # Casting all features in features2 as integers.
        train_df[feature] = train_df[feature].astype(int)
    return train_df

In [17]:
# list of features/float colums to pass into this function
features2 = ['garage_yr_blt', 'lot_frontage', 'mas_vnr_area', 'bsmtfin_sf_1', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf', 'bsmt_full_bath']
float_to_int(features2)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,...,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,69,13517,Pave,IR1,Lvl,AllPub,...,44,0,0,0,0,0,3,2010,WD,130500
1,544,531379050,60,RL,43,11492,Pave,IR1,Lvl,AllPub,...,74,0,0,0,0,0,4,2009,WD,220000
2,153,535304180,20,RL,68,7922,Pave,Reg,Lvl,AllPub,...,52,0,0,0,0,0,1,2010,WD,109000
3,318,916386060,60,RL,73,9802,Pave,Reg,Lvl,AllPub,...,0,0,0,0,0,0,4,2010,WD,174000
4,255,906425045,50,RL,82,14235,Pave,IR1,Lvl,AllPub,...,59,0,0,0,0,0,3,2010,WD,138500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,1587,921126030,20,RL,79,11449,Pave,IR1,HLS,AllPub,...,276,0,0,0,0,0,1,2008,WD,298751
2047,785,905377130,30,RL,69,12342,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,0,3,2009,WD,82500
2048,916,909253010,50,RL,57,7558,Pave,Reg,Bnk,AllPub,...,0,0,0,0,0,0,3,2009,WD,177000
2049,639,535179160,20,RL,80,10400,Pave,Reg,Lvl,AllPub,...,189,140,0,0,0,0,11,2009,WD,144000


In [18]:
# Checking for remaining nulls
null_locations = train_df.columns[train_df.isnull().any()]
train_df[null_locations].isnull().sum()
# NONE shown!

Series([], dtype: float64)

In [19]:
# Looking at unique values of paved_drive column/whether or not it is paved
train_df['paved_drive'].value_counts()

Y    1861
N     151
P      39
Name: paved_drive, dtype: int64

In [20]:
# Using pandas .map() to reassign paved driveways a value of 1, and unpaved/partially paved driveways a value of 0.
train_df['paved_drive'] = train_df['paved_drive'].map({'Y':1, 'N':0, 'P': 0})

In [21]:
# Showing changed with value_counts
train_df['paved_drive'].value_counts()

1    1861
0     190
Name: paved_drive, dtype: int64

In [22]:
# now mapping a value of 1 to houses that have central air, 0 to houses that don't.
train_df['central_air'] = train_df['central_air'].map({'Y':1, 'N':0})

In [63]:
# With some excellent guidance from Kovacs

# Defining a new dictionary, 'map_dict', to hold the keys (currently in features), and the values I to replace them with.
map_dict = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'None':0}

# Defining the list 'ordinals' to contain all the features to pass into my function.
ordinals = ['exter_qual', 'exter_cond', 'bsmt_qual', 'bsmt_cond', 'kitchen_qual', 'fireplace_qu', 'garage_qual', 'garage_cond']

# Defining a function, 'ordinals_to_numeric', converting remaining ordinal columns, ranked on a scale from Poor
# to Excellent, to numeric values. Nulls that I previously converted to the string 'None' will now be represented
# by the number 0.
def ordinals_to_num(train_df):
    # Iterating through features in my training dataframe's columns.
    for feature in train_df.columns:
        # If that feature is in the feature list I defined above as 'ordinals':
        if feature in ordinals:
            # then map the associated numeric dictionary values in 'map_dict' to the keys (found in the original ordinal columns).
            train_df[feature] = train_df[feature].map(map_dict)
            # Converting all features in my list 'ordinals' to numeric values, and coercing errors to return nulls for uninterpretable values. 
            pd.to_numeric(arg=train_df[feature], errors="coerce")
    return train_df

In [64]:
# Applying the function to my dataframe and confirming.
train_df = ordinals_to_num(train_df)
train_df.head(5)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,...,garage_finish,garage_cars,garage_area,paved_drive,wood_deck_sf,open_porch_sf,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,69,13517,Pave,IR1,Lvl,AllPub,...,RFn,2.0,475.0,NaN,0,44,3,2010,WD,130500
1,544,531379050,60,RL,43,11492,Pave,IR1,Lvl,AllPub,...,RFn,2.0,559.0,NaN,0,74,4,2009,WD,220000
2,153,535304180,20,RL,68,7922,Pave,Reg,Lvl,AllPub,...,Unf,1.0,246.0,NaN,0,52,1,2010,WD,109000
3,318,916386060,60,RL,73,9802,Pave,Reg,Lvl,AllPub,...,Fin,2.0,400.0,NaN,100,0,4,2010,WD,174000
4,255,906425045,50,RL,82,14235,Pave,IR1,Lvl,AllPub,...,Unf,2.0,484.0,NaN,0,59,3,2010,WD,138500


In [25]:
# Casting 'ms_subclass' into a string type object.
train_df['ms_subclass'] = train_df['ms_subclass'].astype(str)

In [26]:
# Final check before moving on ...all data types are as expected and desired.
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 77 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2051 non-null   int64  
 1   pid              2051 non-null   int64  
 2   ms_subclass      2051 non-null   object 
 3   ms_zoning        2051 non-null   object 
 4   lot_frontage     2051 non-null   int32  
 5   lot_area         2051 non-null   int64  
 6   street           2051 non-null   object 
 7   lot_shape        2051 non-null   object 
 8   land_contour     2051 non-null   object 
 9   utilities        2051 non-null   object 
 10  lot_config       2051 non-null   object 
 11  land_slope       2051 non-null   object 
 12  neighborhood     2051 non-null   object 
 13  condition_1      2051 non-null   object 
 14  condition_2      2051 non-null   object 
 15  bldg_type        2051 non-null   object 
 16  house_style      2051 non-null   object 
 17  overall_qual  

---
# Section 3

## EDA

In [27]:
# Using .describe to get basic statistics and metrics.
train_df.describe()

,id,pid,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,exter_qual,...,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,saleprice
count,2051.000000,2.051000e+03,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,...,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000
mean,1474.033642,7.135900e+08,69.046319,10065.208191,6.112140,5.562165,1971.708922,1984.190151,98.626524,3.406143,...,93.833740,47.556802,22.571916,2.591419,16.511458,2.397855,51.574354,6.219893,2007.775719,181469.701609
std,843.980841,1.886918e+08,21.306367,6742.488909,1.426271,1.104497,30.177889,21.036250,174.324690,0.587962,...,128.549416,66.747241,59.845110,25.229615,57.374204,37.782570,573.393985,2.744736,1.312014,79258.659352
min,1.000000,5.263011e+08,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,753.500000,5.284581e+08,60.000000,7500.000000,5.000000,5.000000,1953.500000,1964.500000,0.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129825.000000
50%,1486.000000,5.354532e+08,69.000000,9430.000000,6.000000,5.000000,1974.000000,1993.000000,0.000000,3.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,162500.000000
75%,2198.000000,9.071801e+08,78.000000,11513.500000,7.000000,6.000000,2001.000000,2004.000000,159.000000,4.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,2930.000000,9.241520e+08,313.000000,159000.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5.000000,...,1424.000000,547.000000,432.000000,508.000000,490.000000,800.000000,17000.000000,12.000000,2010.000000,611657.000000


In [28]:
# Getting correlation of features to target: SalePrice
corr = train_df.corr()
corr.sort_values(["saleprice"], ascending = False, inplace = True)
saleprice_corr = print(corr.saleprice)

saleprice          1.000000
overall_qual       0.800207
exter_qual         0.712146
gr_liv_area        0.697038
kitchen_qual       0.692336
garage_area        0.650246
garage_cars        0.648197
total_bsmt_sf      0.628670
1st_flr_sf         0.618486
bsmt_qual          0.612188
year_built         0.571849
year_remod/add     0.550370
fireplace_qu       0.538925
full_bath          0.537969
garage_yr_blt      0.518242
totrms_abvgrd      0.504014
mas_vnr_area       0.503579
fireplaces         0.471093
bsmtfin_sf_1       0.423346
open_porch_sf      0.333476
wood_deck_sf       0.326490
lot_frontage       0.325814
lot_area           0.296566
paved_drive        0.289210
garage_qual        0.285281
bsmt_full_bath     0.283332
half_bath          0.283001
central_air        0.277378
garage_cond        0.265021
2nd_flr_sf         0.248452
bsmt_cond          0.225075
bsmt_unf_sf        0.190133
bedroom_abvgr      0.137067
screen_porch       0.134581
3ssn_porch         0.048732
exter_cond         0

---
## Section 4
### Feature Engineering and Dummification

#### -I actually correlated multiple features and did dummify some columns, but found that actually had a negative effect on my model. (Not what I expected)

#### -Unfortunately, I wrote over that code with this dataframe and no longer have it...Below is what I had remaining. Luckily, it wasn't serving me much in this set, anyway.

In [29]:
# viewing corrrelation between saleprice, overall quality, exterior quality and ground living area(sq ft.)
train_df.corr()[['saleprice', 'overall_qual', 'exter_qual', 'gr_liv_area']].sort_values(by='saleprice', ascending=False).head()

,saleprice,overall_qual,exter_qual,gr_liv_area
saleprice,1.000000,0.800207,0.712146,0.697038
overall_qual,0.800207,1.000000,0.740257,0.566701
exter_qual,0.712146,0.740257,1.000000,0.435121
gr_liv_area,0.697038,0.566701,0.435121,1.000000
kitchen_qual,0.692336,0.690639,0.730562,0.444301


In [67]:
dummy_train = ordinals_to_num(train_df)

In [68]:
dummy_train['central_air'] = dummy_train['central_air'].map({'Y':1, 'N':0})
dummy_train['paved_drive'] = dummy_train['paved_drive'].map({'Y':1, 'N':0, 'P':0})

In [69]:
dummy_train.columns

Index(['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'street', 'lot_shape', 'land_contour', 'utilities', 'lot_config',
       'land_slope', 'neighborhood', 'condition_1', 'condition_2', 'bldg_type',
       'house_style', 'overall_qual', 'year_built', 'year_remod/add',
       'roof_style', 'roof_matl', 'exterior_1st', 'exterior_2nd',
       'mas_vnr_type', 'mas_vnr_area', 'exter_qual', 'foundation',
       'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1', 'bsmtfin_type_2',
       'total_bsmt_sf', 'heating', 'heating_qc', 'central_air', 'electrical',
       '1st_flr_sf', '2nd_flr_sf', 'gr_liv_area', 'bsmt_full_bath',
       'full_bath', 'half_bath', 'kitchen_qual', 'totrms_abvgrd', 'functional',
       'fireplaces', 'fireplace_qu', 'garage_type', 'garage_yr_blt',
       'garage_finish', 'garage_cars', 'garage_area', 'paved_drive',
       'wood_deck_sf', 'open_porch_sf', 'mo_sold', 'yr_sold', 'sale_type',
       'saleprice'],
      dtype='object')

In [70]:
dummy_nulls = dummy_train.columns[dummy_train.isnull().any()]
dummy_train[dummy_nulls].isnull

<bound method DataFrame.isnull of       exter_qual  central_air  kitchen_qual  fireplace_qu  paved_drive
0            NaN          NaN           NaN           NaN          NaN
1            NaN          NaN           NaN           NaN          NaN
2            NaN          NaN           NaN           NaN          NaN
3            NaN          NaN           NaN           NaN          NaN
4            NaN          NaN           NaN           NaN          NaN
...          ...          ...           ...           ...          ...
2046         NaN          NaN           NaN           NaN          NaN
2047         NaN          NaN           NaN           NaN          NaN
2048         NaN          NaN           NaN           NaN          NaN
2049         NaN          NaN           NaN           NaN          NaN
2050         NaN          NaN           NaN           NaN          NaN

[2051 rows x 5 columns]>

In [71]:
dummy_train['kitchen_qual'].unique()

array([nan])

## MORE EDA

In [30]:
## MIGHT NOT BE NEEDED, HAS BEEN RUN ON THIS KERNEL
features = ['mas_vnr_type', 'bsmt_qual', 'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_type_2',
           'fireplace_qu', 'garage_type', 'garage_finish', 'garage_qual', 'garage_cond']

In [31]:
null_locations = train_df.columns[train_df.isnull().any()]
train_df[null_locations].isnull().sum()

Series([], dtype: float64)

In [32]:
train_df.head(3)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,...,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,69,13517,Pave,IR1,Lvl,AllPub,...,44,0,0,0,0,0,3,2010,WD,130500
1,544,531379050,60,RL,43,11492,Pave,IR1,Lvl,AllPub,...,74,0,0,0,0,0,4,2009,WD,220000
2,153,535304180,20,RL,68,7922,Pave,Reg,Lvl,AllPub,...,52,0,0,0,0,0,1,2010,WD,109000


In [33]:
train_df.fillna(train_df.mean(), inplace=True)

In [34]:
# function to turn float features to integers
def flt_to_int(features2):
    for feature in features2:
        train_df[feature] = train_df[feature].astype(int)
    return train_df

In [35]:
train_df['saleprice'].head()

0    130500
1    220000
2    109000
3    174000
4    138500
Name: saleprice, dtype: int64

In [36]:
features2 = ['garage_yr_blt', 'lot_frontage', 'mas_vnr_area', 'bsmtfin_sf_1', 'bsmtfin_sf_2', 'bsmt_unf_sf', 
             'total_bsmt_sf', 'bsmt_full_bath']
flt_to_int(features2)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,...,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,69,13517,Pave,IR1,Lvl,AllPub,...,44,0,0,0,0,0,3,2010,WD,130500
1,544,531379050,60,RL,43,11492,Pave,IR1,Lvl,AllPub,...,74,0,0,0,0,0,4,2009,WD,220000
2,153,535304180,20,RL,68,7922,Pave,Reg,Lvl,AllPub,...,52,0,0,0,0,0,1,2010,WD,109000
3,318,916386060,60,RL,73,9802,Pave,Reg,Lvl,AllPub,...,0,0,0,0,0,0,4,2010,WD,174000
4,255,906425045,50,RL,82,14235,Pave,IR1,Lvl,AllPub,...,59,0,0,0,0,0,3,2010,WD,138500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,1587,921126030,20,RL,79,11449,Pave,IR1,HLS,AllPub,...,276,0,0,0,0,0,1,2008,WD,298751
2047,785,905377130,30,RL,69,12342,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,0,3,2009,WD,82500
2048,916,909253010,50,RL,57,7558,Pave,Reg,Bnk,AllPub,...,0,0,0,0,0,0,3,2009,WD,177000
2049,639,535179160,20,RL,80,10400,Pave,Reg,Lvl,AllPub,...,189,140,0,0,0,0,11,2009,WD,144000


In [37]:
null_locations = train_df.columns[train_df.isnull().any()]
train_df[null_locations].isnull().sum()

Series([], dtype: float64)

In [38]:
train_df.shape

(2051, 77)

In [72]:
train_df['saleprice'].head()

0    130500
1    220000
2    109000
3    174000
4    138500
Name: saleprice, dtype: int64

In [40]:
# Log transform the target for official scoring
# train_df.saleprice = np.log1p(train_df.saleprice)
# y = train_df.saleprice
## Turns saleprice to float? couldn't figure out why

In [42]:
train_df['paved_drive'].value_counts()

1    1861
0     190
Name: paved_drive, dtype: int64

In [73]:
# Changing central air and paved drive values to 0s an 1s
train_df['central_air'] = train_df['central_air'].map({'Y':1, 'N':0})
train_df['paved_drive'] = train_df['paved_drive'].map({'Y':1, 'N':0, 'P':0})

In [74]:
map_dict = {"Ex":5, "Gd":4, "TA":3, "Fa":2, "Po":1, "NA":0}
ordinals = ["exter_qual", "exter_cond", "bsmt_qual", "bsmt_cond", "kitchenqual", "fireplacequ", "garage_qual", "garage_cond"]


In [75]:
# Changing year built to include year built-remodel/addition, as they can be the same per data info
train_df['year_built'] = (train_df['year_built'] - train_df['year_remod/add'])

In [76]:
train_df['saleprice'].head()

0    130500
1    220000
2    109000
3    174000
4    138500
Name: saleprice, dtype: int64

In [49]:
null_locations = train_df.columns[train_df.isnull().any()]
train_df[null_locations].isnull().sum()

central_air    2051
paved_drive    2051
dtype: int64

In [51]:
train_df.drop(columns=['enclosed_porch', 'kitchen_abvgr', 'bedroom_abvgr', 'low_qual_fin_sf','misc_val', '3ssn_porch', 
                       'bsmt_half_bath', 'screen_porch', 'pool_area', 'garage_cond', 'bsmt_unf_sf', 'garage_qual', 'bsmt_unf_sf', 'bsmt_qual', 'exter_cond', 'overall_cond',
                      'bsmt_cond', 'bsmtfin_sf_2'], inplace=True)

# - Dropping columns with most Null Value -- cred to Sidni

In [52]:
train_df['saleprice'].head()

0    130500
1    220000
2    109000
3    174000
4    138500
Name: saleprice, dtype: int64

In [53]:
train_df.isnull().sum()

id                   0
pid                  0
ms_subclass          0
ms_zoning            0
lot_frontage         0
lot_area             0
street               0
lot_shape            0
land_contour         0
utilities            0
lot_config           0
land_slope           0
neighborhood         0
condition_1          0
condition_2          0
bldg_type            0
house_style          0
overall_qual         0
year_built           0
year_remod/add       0
roof_style           0
roof_matl            0
exterior_1st         0
exterior_2nd         0
mas_vnr_type         0
mas_vnr_area         0
exter_qual           0
foundation           0
bsmt_exposure        0
bsmtfin_type_1       0
bsmtfin_sf_1         0
bsmtfin_type_2       0
total_bsmt_sf        0
heating              0
heating_qc           0
central_air       2051
electrical           0
1st_flr_sf           0
2nd_flr_sf           0
gr_liv_area          0
bsmt_full_bath       0
full_bath            0
half_bath            0
kitchen_qua

In [62]:
train_df['saleprice'].mean() #Average Sale Price

181469.70160897123

In [ ]:
train_df.head()

In [ ]:
train_df_
= train_df

In [ ]:
# Find most important features relative to target
print("Most Important Features Relative to Sale Price")
corr = train_df.corr()
corr.sort_values(["saleprice"], ascending = False, inplace = True)
saleprice_corr = print(corr.saleprice)

In [ ]:
var = "overall_qual"
data = pd.concat([train_df['saleprice'], train_df[var]], axis=1)
data.plot.scatter(x=var, y='saleprice');
plt.title("Overall Quality and Sale Price")
plt.xlabel("Overall Quality")
plt.ylabel("Sale Price(USD)");

In [ ]:
var = "overall_qual"
data = pd.concat([train_df['saleprice'], train_df[var]], axis=1)
data.plot.scatter(x=var, y='saleprice');

In [ ]:
plt.title("title")
plt.xlabel("x-label")
plt.ylabel("y-label")

In [ ]:
# Looking for outliers
fig.figsize=(16,9)
var = "gr_liv_area"
data = pd.concat([train_df['saleprice'], train_df[var]], axis=1)
data.plot.scatter(x=var, y='saleprice'), ;
plt.title('Above Ground Living Area vs. Price')
plt.xlabel('Above Grade(Ground) Living Area in Sq. Ft')
plt.ylabel('Sale Price(USD)');
plt.xticks(rotation=90, fontsize=6);

In [ ]:
# REMOVING Those outliers where total square footage is greater than 4,500 sq ft
train_df = train_df[train_df['gr_liv_area'] < 4500]
train_df.shape

In [ ]:
# Messing around with some graphs, trying to find a better way of displaying
fig.figsize=(16,9)
sns.barplot(x = train_df['gr_liv_area'], y = train_df['saleprice'])
plt.title("Year Remodeled or Addition vs. Sale Price")
plt.xlabel("Year Remodeled or Addition 1950-2010")
plt.ylabel("Sale Price")
plt.xticks(rotation=90, fontsize=6);

In [ ]:
plt.figure(figsize=(10, 50))

# Using a sorted heatmap to see correlation of features with the sale price.
sns.heatmap(train_df.corr()[['saleprice']].sort_values(by='saleprice', ascending=False),
            # Setting a min and max
            vmin = -1, vmax=1,
            # Choosing a color
            cmap ='BuPu',
            # Annotating each cell
            annot = True,);

### Thoughts

#### I believe I will be able to closely predict prices for the Ames, IA dataset and determine what features cause homes to be higher/lower in value

---
## Section 6

### Exporting cleaned dataframe to new csv

In [56]:
# Dummification first
train_df_filtered = pd.get_dummies(train_df)
train_df_filtered.dropna(inplace=True)

In [ ]:
# SAVED TO CSV
train_df_filtered.to_csv("train_clean.csv") 

In [82]:
train_df.columns

Index(['Id', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
       'G